# AUTOGRAD: AUTOMATIC DIFFERENTIATION

## 参考资料

> https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py  
> https://atomicoo.com/technology/pytorch-auto-diff-autograd/  


In [1]:
import torch
print(torch.__version__)

torch.random.seed()

1.5.0


1124283013989400

## 自动求导

AutoGrad包是PyTorch中所有神经网络的核心，为张量上的所有操作提供自动求导。  
它是一个运行时定义的框架，即反向传播是随着对张量的操作来逐步决定的，这也意味着在每个迭代中都可以是不同的。  

### 张量

`torch.Tensor` 类的重要属性/方法：
  - dtype:   该张量存储的值类型，可选类型见：`torch.dtype`；
  - device:  该张量存放的设备类型，cpu/gpu
  - data:    该张量节点存储的值；
  - requires_grad: 表示autograd时是否需要计算此tensor的梯度，默认False；
  - grad:    存储梯度的值，初始为None；
  - grad_fn: 反向传播时，用来计算梯度的函数；
  - is_leaf: 该张量节点在计算图中是否为叶子节点；

In [2]:
# 将`.requires_grad`设置为true，追踪Tensor上的所有操作
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
# `.grad_fn`属性指向Function，编码Tensor间的运算，
# 包含`.forward()`和`.backward()`
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [5]:
z = y * y * 3
print(z)
out = z.mean()
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


In [6]:
# 就地改变`.requires_grad`属性
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [7]:
# 通过调用`.detach()`阻止追踪Tensor操作（历史记录）
# 或通过将代码块包裹在`with torch.no_grad():`中（评估模型时常用）
# 否则所有Tensor操作（包括拷贝、填充等）都会被追踪
a.detach().fill_(9.)
print(a)
# or
with torch.no_grad():
    a[:] = 11.
print(a)

tensor([[9., 9.],
        [9., 9.]], requires_grad=True)
tensor([[11., 11.],
        [11., 11.]], requires_grad=True)


### 梯度

关于`.backward()`为什么需要参数`grad_tensor`的存在，见下。具体参考 https://zhuanlan.zhihu.com/p/83172023  
扩展：CLASS `torch.autograd.Function` https://pytorch.org/docs/stable/autograd.html#function

In [8]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

In [9]:
# 调用`torch.backward()`自动反向传播计算梯度，并将梯度累积到`.grad`属性中
# 注意，只有`.requires_grad`与`.is_leaf`同时为True的Tensor才会累积梯度
out.backward(create_graph=True)
# 此处`out.backward()`等价于`out.backward(torch.tens or(1.))`
# 因为out为1x1张量，所以`.backward()`参数可以省略
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]], grad_fn=<CloneBackward>)


In [ ]:
# 自动计算梯度还可以使用以下方法调用
torch.autograd.backward(out, create_graph=True)
print(x.grad)
# or
torch.autograd.backward(out, create_graph=True)
print(x.grad)

In [10]:
x = torch.ones(2,requires_grad=True)
z = x + 2
z.backward(torch.ones_like(z))
print(x.grad)

tensor([1., 1.])


In [11]:
# 由于反向传播中梯度是进行累积的，所以当输出out为矩阵时
# 直接反向计算梯度与先将out元素求和再反向计算梯度的结果是一样的
# 通过设置`grad_tensor`参数可以“加权”求和
x = torch.ones(2,requires_grad=True)
z = x + 2
z.sum().backward()
print(x.grad)

tensor([1., 1.])


数学上，若给定 $\vec{y}=f(\vec{x})$，则 $\vec{y}$ 对 $\vec{x}$ 的梯度是一个 Jacobian 矩阵：

$$
\begin{split}J=\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)
\end{split}
$$

若标量函数 $l=g(\vec{y})$ 的梯度为 $v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$，则 $l$ 对 $\vec{x}$ 的梯度是如下 vector-Jacobian product：

$$
\begin{split}J^{T}\cdot v=\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)\left(\begin{array}{c}
 \frac{\partial l}{\partial y_{1}}\\
 \vdots\\
 \frac{\partial l}{\partial y_{m}}
 \end{array}\right)=\left(\begin{array}{c}
 \frac{\partial l}{\partial x_{1}}\\
 \vdots\\
 \frac{\partial l}{\partial x_{n}}
 \end{array}\right)
\end{split}
$$

`torch.autograd` 就是一个计算 vector-Jacobian product 的引擎。


In [15]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([1680.4718, -246.4453,  961.9392], grad_fn=<MulBackward0>)


In [16]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])
